# 副問い合わせ & テーブルの定義と更新

In [1]:
import os

if not os.path.isfile('data/SSDSE.db'):
    os.makedirs('data', exist_ok=True)

    if not os.path.isfile('download_SSDSE.py'):
        !wget https://raw.githubusercontent.com/hontolab-courses/database-lecturenote/main/content/sql/download_SSDSE.py
        import download_SSDSE

In [2]:
!pip install jupysql

In [3]:
%load_ext sql
%config SqlMagic.feedback = 0
%config SqlMagic.displaylimit = 50
%sql sqlite:///data/SSDSE.db

前章，前々章同様，本章ではSQLの説明のために独立行政法人統計センターが公開している教育用標準データセット（SSDSE）の[基本素材SSDSE-E](https://www.nstac.go.jp/use/literacy/ssdse/#SSDSE-E)（データの解説は[こちら](https://www.nstac.go.jp/sys/files/kaisetsu-E-2023.pdf)）から抜粋・加工したデータ（`population`テーブル）を用いる．
`population`テーブルには，47ある各都道府県に関する総人口，小学校児童数，中学校生徒数，高等学校生徒数，大学学生数のデータが2021年度，2020年度分入っている．
このテーブルが格納された関係データベースが手元にあると想定して，SQLの使い方を説明する．

## 副問い合わせ

関係データベースへの問い合わせが複雑になってくると，SQLによる問い合わせ結果を使ってさらにSQL文を書きたいケースも発生する．
このようなケースで役に立つのが**副問い合わせ（サブクエリ; subqueryと呼ぶことも）** である．
副問い合わせを用いると，`FROM`句や`WHERE`句の中で`SELECT`文を書くことができ，SQL文の中で別のSQL文の実行結果を参照することができる．

例題を通して副問い合わせの使い方を確認しよう．
`population`テーブルを用いて
1. 2021年度と2020年度の大学学生数の平均値を都道府県別に算出し，
2. 平均大学生数の上位10の都道府県を表示する

SQL文について考えたい．
上記ステップ1だけを行うSQL文であれば，以下のようになる．

```sql
/* ステップ1だけを行うSQL文 */
SELECT
  地域コード,
  都道府県,
  AVG(大学学生数) AS 平均大学生数 --- 列名に名前をつける
FROM
  population
GROUP BY
  都道府県; 
```

In [4]:
%%sql
    
/* ステップ1だけを行うSQL文 */
SELECT
  地域コード,
  都道府県,
  AVG(大学学生数) AS 平均大学生数 --- 列名に名前をつける
FROM
  population
GROUP BY
  都道府県;

地域コード,都道府県,平均大学生数
R24000,三重県,14063.0
R26000,京都府,142482.5
R41000,佐賀県,7758.5
R28000,兵庫県,115917.5
R01000,北海道,79569.0
R12000,千葉県,105451.5
R30000,和歌山県,7707.5
R11000,埼玉県,110676.0
R44000,大分県,15233.5
R27000,大阪府,227323.0


今やりたいことは，この結果テーブルを平均大学生数で大きい順に並べ替えて，上位10件の都道府県レコードのみを出力することである．
仮に上記結果テーブルが`population_avg`という名で存在していれば，以下のようなSQL文を書くことで目的を達成できる．

```sql
--- これは仮のSQL文
SELECT * FROM population_avg ORDER BY 平均大学生数 DESC LIMIT 10;
```

上記のようなことは，ステップ1のSQL文を「副問い合わせ指定」することで実現できる．
具体的には，上記仮のSQL文の`FROM`句にある`population_avg`の箇所について，下記のSQL文のようにカッコで囲んだ「ステップ1だけを行う」`SELECT`文で置き換える．

```sql
SELECT
  *
FROM
  (
    SELECT
      地域コード,
      都道府県,
      AVG(大学学生数) AS 平均大学生数
    FROM
      population
    GROUP BY
      都道府県
  ) 
ORDER BY
  population_avg.平均大学生数 DESC
LIMIT 10; 
```

In [5]:
%%sql
    
SELECT
  *
FROM
  (
    SELECT
      地域コード,
      都道府県,
      AVG(大学学生数) AS 平均大学生数
    FROM
      population
    GROUP BY
      都道府県
  ) AS population_avg -- AS修飾句を使って，副問い合わせの結果テーブルに名前をつける
ORDER BY
  population_avg.平均大学生数 DESC
LIMIT
  10;

地域コード,都道府県,平均大学生数
R13000,東京都,675323.5
R27000,大阪府,227323.0
R23000,愛知県,177051.0
R14000,神奈川県,172937.0
R26000,京都府,142482.5
R28000,兵庫県,115917.5
R11000,埼玉県,110676.0
R40000,福岡県,110008.0
R12000,千葉県,105451.5
R01000,北海道,79569.0


副問い合わせを用いることで，SQL文の実行結果を別のSQL文の中で参照することができる．
なお，副問い合わせはSQL文が複雑で読みにくくなる．
そのため，少しでもわかりやすくするために，副問い合わせ箇所に`AS`修飾句で名前をつけたり，`WITH`句を使って副問い合わせ箇所を外出しすることができる．

以下のSQL文は，上記SQL文を`AS`修飾句や`WITH`句を使ってわかりやすく書き換えたものである．

```sql
/* 分かりやすく書き換えたSQL1 */
SELECT
  *
FROM
  (
    SELECT
      地域コード,
      都道府県,
      AVG(大学学生数) AS 平均大学生数
    FROM
      population
    GROUP BY
      都道府県
  ) AS population_avg -- AS修飾句を使って，副問い合わせの結果テーブルに名前をつける
ORDER BY
  population_avg.平均大学生数 DESC
LIMIT 10;

```sql
/* 分かりやすく書き換えたSQL2 */
/* WITH句を使って，副問い合わせ箇所をメインのSQL文の外に出している */
WITH population_avg AS (
    SELECT
      地域コード,
      都道府県,
      AVG(大学学生数) AS 平均大学生数
    FROM
      population
    GROUP BY
      都道府県    
)
SELECT
  *
FROM
  population_avg
ORDER BY
  population_avg.平均大学生数 DESC
LIMIT 10; 
```

上の例では`FROM`句内で副問い合わせ指定を行っていたが，`WHERE`句においても副問い合わせを用いることができる．
例えば，[前章](../02.ipynb)の例題で用いた
- 65歳以上人口数の上位10都道府県のリストが格納された`elderly_population_top10`テーブル
- 「学習・自己啓発・訓練」「スポーツ」「趣味・娯楽」「ボランティア」「旅行・行楽」について過去1年以内に活動したことのある人の割合（%）について都道府県別にまとめた`activity`テーブル

の2つを用いて，
> 65歳以上人口数の上位10件に「入っていない」都道府県について，過去1年以内に「旅行・行楽」活動をしたことのある人の割合が5割を超える場合のみ，その割合を地域コード，都道府県名とともに表示する

ためのSQL文を考えてみよう．
ちなみに，
> 65歳以上人口数の上位10件に「**入っている**」都道府県について，過去1年以内に「旅行・行楽」活動をしたことのある人の割合が5割を超える場合のみ，その割合を地域コード，都道府県名とともに表示する

のであれば，以下のようなSQL文になる（[★Quiz1★](#sql3-q1)）．


```sql
SELECT
  activity.地域コード,
  activity.都道府県,
  activity.旅行・行楽
FROM
  activity
  JOIN
    elderly_population_top10
    ON activity.都道府県 = elderly_population_top10.都道府県
  AND activity.旅行・行楽 >= 50;
```

In [6]:
%%sql
    
SELECT
  activity.地域コード,
  activity.都道府県,
  activity.旅行・行楽
FROM
  activity
  JOIN
    elderly_population_top10
    ON activity.都道府県 = elderly_population_top10.都道府県
  AND activity.旅行・行楽 >= 50;

地域コード,都道府県,旅行・行楽
R01000,北海道,51.0
R11000,埼玉県,51.5
R13000,東京都,55.5
R14000,神奈川県,54.5
R23000,愛知県,57.6
R27000,大阪府,51.9
R28000,兵庫県,51.6
R40000,福岡県,52.3


65歳以上人口数の上位10件に「入っている」都道府県のケースとは違って，65歳以上人口数の上位10件に「入っていない」都道府県のケースの場合，
1. `activity`テーブル内の各都道府県について，`elderly_population_top`テーブル内にあるか否かを判定し，
2. ない場合のみ，その都道府県の「旅行・行楽」の値を見る
3. 「旅行・行楽」の値が50を超えるものだけ，地域コード，都道府県名とともに表示する

という流れになる．
上記ステップ1では「別のテーブルを活用した対象レコードの存在判定」を行っているが，存在判定は **`EXISTS`述語** を使った副問い合わせで対応することができる．
上記例の場合，以下のようなSQL文になる．

```sql   
SELECT
  activity.地域コード,
  activity.都道府県,
  activity.旅行・行楽
FROM
  activity
WHERE
　--- NOT EXISTSは存在しない場合Trueを，存在する場合はFalseを返す
  NOT EXISTS (　
    SELECT
      *
    FROM
      elderly_population_top10
    WHERE
      elderly_population_top10.都道府県 = activity.都道府県
  )
  AND activity.旅行・行楽 >= 50; 
```

In [7]:
%%sql
    
SELECT
  activity.地域コード,
  activity.都道府県,
  activity.旅行・行楽
FROM
  activity
WHERE
　--- NOT EXISTSは存在しない場合Trueを，存在する場合はFalseを返す
  NOT EXISTS (
    SELECT
      *
    FROM
      elderly_population_top10
    WHERE
      elderly_population_top10.都道府県 = activity.都道府県
  )
  AND activity.旅行・行楽 >= 50; 

地域コード,都道府県,旅行・行楽
R04000,宮城県,52.2
R10000,群馬県,50.8
R25000,滋賀県,52.2
R29000,奈良県,50.4
R43000,熊本県,50.5


`WHERE`句内で`EXISTS`が用いられた場合，`FROM`句で指定されたテーブルの各レコードについて，
* `EXISTS`以下の副問い合わせ内容を満たすものが**1つでも**存在すれば`True（真）`，
* 存在しなければ`False（偽）`

の値を返す．
`NOT EXISTS`を用いた場合は逆の振る舞いになる．すなわち，
* `NOT EXISTS`以下の副問い合わせ内容を満たすものが**1つでも**存在すれば`False（偽）`，
* 1つも存在しなければ`True（真）`

の値を返す．
`EXISTS`の振る舞いは少しややこしいが，副問い合わせを用いたSQL文では比較的よく用いられるので頭に入れておこう（[★Quiz2★](#sql3-q2)）．

````{note}
#### IN述語
`EXISTS`述語と似たようなものに`IN`述語がある．
`IN`述語は`WHERE`句内で`列名 IN リスト`と書くと，あるレコードの列の値が`リスト`内にあるかどうかをTrue or Falseで返す．
なお，`リスト`と書かれたところは`(値, 値, 値)`のようにベタ書きで指定してもよいし，副問い合わせでも指定してもよい．

例えば，`NOT EXISTS`述語を用いた先の例のSQL文は`IN`述語を使って以下のように書き換えられる．
```sql
SELECT
  activity.地域コード,
  activity.都道府県,
  activity.旅行・行楽
FROM
  activity
WHERE
　--- activity.都道府県の値がelderly_population_top10.都道府県のリストになければTrueを返す
  activity.都道府県 NOT IN (
    SELECT
      elderly_population_top10.都道府県
    FROM
      elderly_population_top10
  )
  AND activity.旅行・行楽 >= 50; 
```

`EXISTS`述語よりも`IN`述語の方が挙動が分かりやすいが，実行速度は`EXISTS`述語の方が速い．
`IN`を用いた場合，副問い合わせのSQL文は条件に当てはまるレコードをすべて列挙する．
それに対して`EXISTS`を用いた場合は，条件に当てはまるレコードが1つでも見つかれば処理を終了して次の処理に移るので速い．
````

```{tip}
#### 問い合わせのための発展的なSQL
これまでに述べてきたSQLは，関係データベースから所望のレコードを引っ張ってくるためのあくまで基礎である．
解説していないSQLは他にも色々ある．
例えば，
* 条件分岐をするための`CASE`句
* 数値，文字列処理のための様々な関数
* より高度な集約演算を行うためのWindow関数（分析関数）

などがある．
興味のある方は
- 本橋智光（著）. 2018. [前処理大全-データ分析のためのSQL/R/Python実践テクニック](https://gihyo.jp/book/2018/978-4-7741-9647-3). 技術評論社
- 加嵜長門, 田宮直人（著）. 2017. [ビッグデータ分析・活用のためのSQLレシピ](https://book.mynavi.jp/ec/products/detail/id=65863). マイナビ出版

など，実践的かつ面白い書籍があるので探して読んでみるとよい．
```

## テーブルの定義と更新

SQLの締めくくりとして，関係データベースにおけるテーブルの定義と更新操作（挿入．削除．更新）について説明する．

### テーブルの定義

SQLでテーブルの定義を行うには`CREATE TABLE`文を用いる．
テーブル定義では
* テーブル名
* テーブルが持つ列名
* 各列のデータ型
* （必要なら）主キー，外部キー
* （必要なら）整合性制約

を指定する．

データ型は[関係データモデル](../relational-data-model/01.ipynb)の章で説明した**定義域（domain）** に対応するものである．
関係データベースで扱える代表的なデータ型は以下の通りである．
* `VARCHAR(n)`: n文字まで扱える可変長の文字列型．通常文字列を扱うときはこれを指定する．
* `INT`: 整数型
* `FLOAT`: 浮動小数点型
* `BOOLEAN`: 真偽値型（TRUEもしくはFALSEの値をもつ）
* `DATETIME`: 時刻を併せ持つ日付型

例えば，例題で用いてきた`population`テーブルをゼロから定義する場合，以下のようなSQL文となる．

```sql
CREATE TABLE population ( --- TABLEの横にテーブル名を書く
    地域コード VARCHAR(10),　--- 10文字あれば地域コードを網羅的に表現できるので10を指定
    都道府県 VARCHAR(5), --- 5文字あれば都道府県を網羅的に表現できるので5を指定
    調査年度 INT,
    総人口 INT,
    小学校児童数 INT,
    中学校生徒数 INT,
    高等学校生徒数 INT,
    大学学生数 INT,
    PRIMARY KEY (地域コード, 調査年度)　 --- 主キーとして地域コードを指定
);
```

なお，上記は最低限の説明である．
実際にSQL文を使ってテーブル定義をする際には，初期値や外部キー制約，インデックスなど，想定するアプリケーションに応じて様々な設定を行うことになる．
使用する関係データベース管理システムによって微妙に設定方法が異なるので，RDBMSに対応したドキュメントを適宜参照すること．

### レコードの追加

レコードの追加には`INSERT INTO`文を用いる．
以下は，`population`テーブルに2つのレコードを追加するSQL文の例である．

```sql
INSERT INTO
  population ( --- ここに並べた列名に対応した値をVALUES以下のカッコ内に書く．
    地域コード,
    都道府県,
    調査年度,
    総人口,
    小学校児童数,
    中学校生徒数,
    高等学校生徒数,
    大学学生数
  )
VALUES 
--- VALUES以下に追加したいレコード数分だけ値をカッコで並べる．カッコ内の順番は列名の順序に対応
--- xxxは適当な値
  ('R01000', '北海道', '2022', xxx, xxx, xxx, xxx, xxx),
  ('R01000', '北海道', '2023', xxx, xxx, xxx, xxx, xxx);
```

### レコードの削除

レコードの削除には`DELETE`文を用いる．
以下は，`population`テーブルから調査年度が2022もしくは2023のレコードを削除するSQL文の例である．

```sql
DELETE FROM
  population
WHERE
  調査年度 = 2022 OR 調査年度 = 2023; -- SELECT分と同様に`WHERE`句で条件を指定できる
```

### レコードの更新

レコードの更新には`UPDATE`文を用いる．
以下は，`population`テーブルから調査年度が2022のレコードについて，総人口の値更新前の総人口から100増やしたものに更新するSQL文である．

```sql
UPDATE 
  population
SET
  総人口 = 総人口 + 100
WHERE
  調査年度 = 2022; -- SELECT分と同様に`WHERE`句で条件を指定できる
```

---

## クイズ

本クイズでは，独立行政法人統計センターが公開している教育用標準データセット（SSDSE）の[基本素材SSDSE-E](https://www.nstac.go.jp/use/literacy/ssdse/#SSDSE-E)（データの解説は[こちら](https://www.nstac.go.jp/sys/files/kaisetsu-E-2023.pdf)）および[市区町村（SSDSE-A）](https://www.nstac.go.jp/use/literacy/ssdse/#SSDSE-A)から抜粋・加工したデータを用いる．
なお，                                                           
- `population`テーブルの内容については[前々章](./01.ipynb)
- `activity`テーブルの内容については[前章](./02.ipynb)                                                           

を参照せよ．


(sql3-q1)=
### Q1. 副問い合わせ（1/2）

`population`テーブルにおいて調査年度が`2021`時点の小学校児童数上位20件のレコードを求め，それらと`activity`テーブルの内容を結合した内容を表示するSQL文を書け．

In [8]:
%%sql



(sql3-q2)=
### Q2. 副問い合わせ（2/2）

`activity`テーブルにおいて「ボランティア」の値が上位10件以内に入る都道府県を求めるSQL文を考えよ．
その上で，`population`テーブルを用いて，「ボランティア」の値が上位10件以内に入る各都道府県の調査期間（2020および2021）における平均総人口を求めるSQL文を書け．


In [9]:
%%sql

